# Age of Empires 2 player stats

## Retrieve the data from the api

In [1]:
import requests
import numpy as np
import pandas as pd

n = 1000

profile_id = 76561198141504553

resp_games = requests.get('https://aoe2.net/api/player/matches?game=aoe2de&steam_id=' + str(profile_id) + '&count=' + str(n)).json()
resp_games = [i for i in resp_games if ((i.get('num_players') == 2) & (i.get('name') == 'AUTOMATCH'))]

resp_ratings = requests.get('https://aoe2.net/api/player/ratinghistory?game=aoe2de&leaderboard_id=3&steam_id=' + str(profile_id) + '&count=' + str(n)).json()
resp_ratings = resp_ratings[:len(resp_games)]
resp_string = requests.get('https://aoe2.net/api/strings').json()
civs_str = resp_string.get('civ')
maps_str = resp_string.get('map_type')

players = [i.get('players') for i in resp_games]
players = [j for i in players for j in i]
players = [i for i in players if i.get('steam_id') == str(profile_id)]

other_players = [i.get('players') for i in resp_games]
other_players = [j for i in other_players for j in i]
other_players = [i for i in other_players if i.get('steam_id') != str(profile_id)]

civs = [i.get('civ') for i in players]
l_civs = []
for i in civs:
    l_civs.append([j.get('string') for j in civs_str if j.get('id') == i][0])
civs = l_civs

other_civs = [i.get('civ') for i in other_players]
l_civs = []
for i in other_civs:
    l_civs.append([j.get('string') for j in civs_str if j.get('id') == i][0])
other_civs = l_civs

maps = [i.get('map_type') for i in resp_games]
l_maps = []
for i in maps:
    l_maps.append([j.get('string') for j in maps_str if j.get('id') == i][0])
maps = l_maps

winlose = [i.get('won') for i in players]

df = pd.DataFrame({'Maps':maps,'Civs':civs,'Win/Lose':winlose,'Other player civ':other_civs})
df['Victories'] = np.where(df['Win/Lose']==1,1,0)
df['Defeats'] = np.where(df['Win/Lose']==0,1,0)
df['Counted games'] = df['Victories'] + df['Defeats']

df_agg = df.groupby(['Civs'])[['Victories','Defeats','Counted games']].sum()
df_agg['Winrate'] = df_agg['Victories'] / (df_agg['Victories'] + df_agg['Defeats'])

df_other_agg = df.groupby(['Other player civ'])[['Victories','Defeats','Counted games']].sum()
df_other_agg['Winrate'] = df_other_agg['Victories'] / (df_other_agg['Victories'] + df_other_agg['Defeats'])

df_maps_agg = df.groupby(['Maps'])[['Victories','Defeats','Counted games']].sum()
df_maps_agg['Winrate'] = df_maps_agg['Victories'] / (df_maps_agg['Victories'] + df_maps_agg['Defeats'])

## Winrate by civs

In [2]:
df_agg = df.groupby(['Civs'])[['Victories','Defeats','Counted games']].sum()
df_agg['Winrate'] = df_agg['Victories'] / (df_agg['Victories'] + df_agg['Defeats'])
df_agg.sort_values(by='Counted games',ascending=False)

,Victories,Defeats,Counted games,Winrate
Civs,,,,
Magyars,6,9,15,0.400000
Vietnamese,7,8,15,0.466667
Vikings,7,8,15,0.466667
Malians,10,3,13,0.769231
Britons,6,4,10,0.600000
Italians,4,5,9,0.444444
Berbers,4,5,9,0.444444
Mayans,3,5,8,0.375000
Slavs,3,3,6,0.500000


## Winrate by civs against

In [3]:
df_other_agg = df.groupby(['Other player civ'])[['Victories','Defeats','Counted games']].sum()
df_other_agg['Winrate'] = df_other_agg['Victories'] / (df_other_agg['Victories'] + df_other_agg['Defeats'])
df_other_agg.sort_values(by='Counted games',ascending=False)

,Victories,Defeats,Counted games,Winrate
Other player civ,,,,
Mongols,7,6,13,0.538462
Persians,6,5,11,0.545455
Aztecs,3,4,7,0.428571
Britons,5,2,7,0.714286
Mayans,3,4,7,0.428571
Huns,4,2,6,0.666667
Cumans,4,2,6,0.666667
Japanese,3,2,5,0.600000
Celts,1,4,5,0.200000


## Winrate by maps

In [4]:
df_maps_agg = df.groupby(['Maps'])[['Victories','Defeats','Counted games']].sum()
df_maps_agg['Winrate'] = df_maps_agg['Victories'] / (df_maps_agg['Victories'] + df_maps_agg['Defeats'])
df_maps_agg.sort_values(by='Counted games',ascending=False)

,Victories,Defeats,Counted games,Winrate
Maps,,,,
Arabia,13,12,25,0.520000
Four Lakes,14,6,20,0.700000
Golden Swamp,8,5,13,0.615385
MegaRandom,3,10,13,0.230769
Serengeti,5,7,12,0.416667
Arena,2,6,8,0.250000
Hill Fort,3,3,6,0.500000
Nomad,3,2,5,0.600000
Acropolis,3,1,4,0.750000
